<a href="https://colab.research.google.com/github/kasubikila/githubTest/blob/main/%3Cyour_x500%3EHomework4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install py3dmol

In [9]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as plt


In [10]:
import numpy as np
from numpy.linalg import norm
# fix  seed  for  reproducible  result. Please  do not  change  the  seed
rng = np.random.default_rng(20232033)
n = 300
A = rng.random((n,n))
b = rng.random((n,1))
#print("A = {}; \n b = {}".format(A,b))

**1.1 (1.5/4) Gaussian Elimination (Version 0)**

**Implement your own function: TODO**

In [ ]:
def gauss_elim_v0(A, b, print_flag=False):
    # Step 1: Concatenate A and b to form the augmented matrix U
    U = np.concatenate((A, b.reshape(-1, 1)), axis=1)
    n = U.shape[0]  # Number of rows

    for k in range(n - 1):  # Iterate from 0 to n-2
        for j in range(k + 1, n):  # Iterate through rows below the k-th row
            # Calculate lambda (scale factor)
            lam = U[j, k] / U[k, k]
            # Update row j
            U[j, :] = U[j, :] - lam * U[k, :]

            if print_flag:
                print(f"Step {k},{j}:\n", U)  # Print intermediate U matrix after each operation

    return U


**Validate your result here

In [12]:
# Test case input
A = np.array([[1, -1, 1],
              [2, -1, 3],
              [2,  0, 3]], dtype=float)  # The coefficient matrix

b = np.array([1, 4, 5], dtype=float)  # The right-hand side vector

# Function call
U = gauss_elim_v0(A, b, print_flag=True)

# Print the final augmented matrix
print("Final Augmented Matrix:\n", U)

Step 1:
[[ 1. -1.  1.  1.]
 [ 0.  1.  1.  2.]
 [ 0.  2.  1.  3.]]
Step 2:
[[ 1. -1.  1.  1.]
 [ 0.  1.  1.  2.]
 [ 0.  0. -1. -1.]]
Final Augmented Matrix:
 [[ 1. -1.  1.  1.]
 [ 0.  1.  1.  2.]
 [ 0.  0. -1. -1.]]


**1.2 (1.5/4) Back Substitution PartA**

In [27]:
import numpy as np

def back_subs(U, print_flag=False):
    """
    Performs back substitution to solve a system of linear equations with an upper triangular matrix.

    Args:
        U: Upper triangular augmented matrix (numpy array).
        print_flag: Boolean flag to control printing intermediate results (default: False).

    Returns:
        x: Solution vector (numpy array).
    """
    n = U.shape[0]  # Number of rows (equations)
    x = np.zeros(n)  # Initialize the solution vector as an all-zero vector
    c = U[:, -1]  # Last column of the augmented matrix (updated b)
    D = U[:, :-1]  # Rest part of the augmented matrix (updated A)

    # Start back substitution from the last row
    x[n - 1] = c[n - 1] / D[n - 1, n - 1]
    if print_flag:
        print(f"R{n-1}: x{n-1} = {x[n-1]}")  # Print the first solved variable

    # Iterate from second-last row up to the first row
    for i in range(n - 2, -1, -1):
        sum_j = np.sum(D[i, i+1:] * x[i+1:])
        x[i] = (c[i] - sum_j) / D[i, i]
        if print_flag:
            print(f"R{i}: x{i} = {x[i]}")  # Print the newly solved variable

    return x

# Test case for back substitution
U_test = np.array([
    [1, -1, 1, 1],
    [0, 1, -1, -1],
    [0, 0, -1, -1]
], dtype=float)

print("Back Substitution Results:")
back_subs(U_test, print_flag=True)

Back Substitution Results:
R2: x2 = 1.0
R1: x1 = 0.0
R0: x0 = 0.0


array([0., 0., 1.])

**1.2 (1.5/4) Back Substitution PartB**


To implement my_solver_v0, we need to combine the Gaussian elimination function (gauss_elim_v0) and the back_subs function. Let's proceed step by step.

Step-by-Step Approach:
Gaussian Elimination (gauss_elim_v0): This function performs Gaussian elimination to convert the augmented matrix
[
𝐴
∣
𝑏
]
[A∣b] into row echelon form.
Back Substitution (back_subs): This function will be used to solve the system once the augmented matrix is in row echelon form.
Combining the Two Functions: The my_solver_v0 function will combine both functions: first performing Gaussian elimination to obtain the upper triangular matrix, then using back substitution to find the solution
𝑥
x.


Full Implementation of my_solver_v0:

In [29]:
import numpy as np

# Gaussian elimination (upper triangular form)
def gauss_elim_v0(A, b):
    """
    Perform Gaussian elimination to convert [A|b] into row echelon form.

    Arguments:
    A -- The matrix (n x n)
    b -- The vector (n x 1)

    Returns:
    U -- The upper triangular matrix [A|b] in row echelon form
    """
    n = A.shape[0]
    U = np.hstack([A, b.reshape(-1, 1)])  # Create augmented matrix [A|b]

    for i in range(n):
        # Partial pivoting: Find the maximum element in the current column
        max_row = np.argmax(np.abs(U[i:n, i])) + i
        U[[i, max_row]] = U[[max_row, i]]  # Swap rows if necessary

        # Eliminate the entries below the pivot
        for j in range(i + 1, n):
            factor = U[j, i] / U[i, i]
            U[j, i:] -= factor * U[i, i:]

    return U  # Return the augmented upper triangular matrix [U|b]

# Back substitution to solve the upper triangular system
def back_subs(U, print_flag=False):
    """
    Perform back substitution to solve the system represented by the augmented matrix [U|b].

    Arguments:
    U -- The upper triangular matrix [U|b] (n x (n+1))
    print_flag -- A flag to control printing the newly solved variables during back substitution.

    Returns:
    x -- The solution vector (n x 1)
    """
    n = U.shape[0]
    x = np.zeros(n)
    c = U[:, -1]  # Last column of the augmented matrix (updated b)
    D = U[:, :-1]  # The matrix part of the augmented matrix (updated A)

    # Start back substitution from the last row
    x[n - 1] = c[n - 1] / D[n - 1, n - 1]
    if print_flag:
        print(f"R{n-1}: x{n-1} = {x[n-1]}")

    # Iterate from second-last row up to the first row
    for i in range(n - 2, -1, -1):
        sum_j = np.sum(D[i, i+1:] * x[i+1:])
        x[i] = (c[i] - sum_j) / D[i, i]
        if print_flag:
            print(f"R{i}: x{i} = {x[i]}")

    return x

# Solve Ax = b using Gaussian Elimination and Back Substitution
def my_solver_v0(A, b):
    """
    Solve the linear system Ax = b using Gaussian Elimination and Back Substitution.

    Arguments:
    A -- The matrix (n x n)
    b -- The vector (n x 1)

    Returns:
    x -- The solution vector (n x 1)
    """
    # Perform Gaussian elimination to get the upper triangular matrix
    U = gauss_elim_v0(A, b)

    # Use back substitution to solve for x
    x = back_subs(U)

    return x

# Test the solver with random data
np.random.seed(0)  # For reproducibility

# Generate a random 300x300 matrix A and a random vector b
A = np.random.rand(300, 300)
b = np.random.rand(300)

# Solve using my_solver_v0
x1 = my_solver_v0(A, b)

# Validate the solution by calculating the relative error
Ax1 = A @ x1  # Matrix-vector multiplication
relative_error = np.linalg.norm(Ax1 - b) / np.linalg.norm(A, 'fro')  # Frobenius norm of A
print(f"Relative error (Ax1 - b) / ||A||_F: {relative_error}")

# Solve using NumPy's built-in solver
x2 = np.linalg.solve(A, b)

# Calculate the relative distance between the two solutions
relative_distance = np.linalg.norm(x1 - x2) / np.linalg.norm(x2)
print(f"Relative distance between x1 and x2: {relative_distance}")


Relative error (Ax1 - b) / ||A||_F: 1.1306485829638964e-15
Relative distance between x1 and x2: 2.483186858092973e-14


**1.3 (1/4) Gaussian Elimination (Version 1)**

In [30]:
import numpy as np

def back_subs(U):
    """
    Performs back substitution to solve a system of linear equations with an upper triangular matrix.

    Args:
        U: Upper triangular augmented matrix (numpy array).

    Returns:
        x: Solution vector (numpy array).
    """
    n = U.shape[0]  # Number of rows (equations)
    x = np.zeros(n)  # Initialize the solution vector as an all-zero vector
    c = U[:, -1]  # Last column of the augmented matrix (updated b)
    D = U[:, :-1]  # Rest part of the augmented matrix (updated A)

    # Start back substitution from the last row
    x[n - 1] = c[n - 1] / D[n - 1, n - 1]

    # Iterate from second-last row up to the first row
    for i in range(n - 2, -1, -1):
        sum_j = np.sum(D[i, i+1:] * x[i+1:])
        x[i] = (c[i] - sum_j) / D[i, i]

    return x

def gauss_elim_v1(A, b, print_flag=False):
    """
    Performs Gaussian elimination with partial pivoting on the augmented matrix [A | b].

    Args:
        A: Square matrix (numpy array).
        b: Right-hand side vector (numpy array).
        print_flag: Boolean flag to control printing intermediate steps (default: False).

    Returns:
        U: Augmented matrix in row-echelon form (numpy array).
    """
    assert A.shape[0] == A.shape[1], "A must be square"
    assert A.shape[0] == b.shape[0], "Input dimension doesn't match"
    assert b.shape[1] == 1, "b is not a vector"

    n = A.shape[0]
    U = np.concatenate((A, b), axis=1)  # Create augmented matrix

    if print_flag:
        print(U)
        print("==============================================")

    for k in range(n - 1):
        # Find the row with the largest absolute value in the current column
        max_row = np.argmax(np.abs(U[k:, k])) + k

        # Swap rows
        U[[k, max_row]] = U[[max_row, k]]

        # Eliminate elements below the pivot
        for j in range(k + 1, n):
            factor = U[j, k] / U[k, k]
            U[j, :] -= factor * U[k, :]

        if print_flag:
            print(U)
            print("==============================================")

    return U

def my_solver_v1(A, b):
    """
    Solves the linear system Ax = b using Gaussian elimination and back substitution.

    Args:
        A: Square matrix (numpy array).
        b: Right-hand side vector (numpy array).

    Returns:
        x: Solution vector (numpy array).
    """
    U = gauss_elim_v1(A, b, False)
    x = back_subs(U)
    return x

# Test case for gauss_elim_v1 (example from Eq. (4))
A_example2 = np.array([[0, 1, 1], [2, 6, 4], [1, 1, 4]], dtype=np.float64)
b_example2 = np.array([[-1], [6], [9]], dtype=np.float64)

U_example2 = gauss_elim_v1(A_example2, b_example2, True)
print(U_example2)

# Test case for my_solver_v1
A = np.random.rand(300, 300)
b = np.random.rand(300, 1)  # Ensure b is a column vector

x1 = my_solver_v1(A, b)
x2 = np.linalg.solve(A, b)

# Calculate relative error and distance
rel_error = np.linalg.norm(A @ x1 - b) / np.linalg.norm(b)
rel_dist = np.linalg.norm(x1 - x2) / np.linalg.norm(x2)

print("The relative error ||Ax1-b||/||b|| = {}".format(rel_error))
print("The relative distance ||x1-x2||/||x2|| = {}".format(rel_dist))

[[ 0.  1.  1. -1.]
 [ 2.  6.  4.  6.]
 [ 1.  1.  4.  9.]]
[[ 2.  6.  4.  6.]
 [ 0.  1.  1. -1.]
 [ 0. -2.  2.  6.]]
[[ 2.  6.  4.  6.]
 [ 0. -2.  2.  6.]
 [ 0.  0.  2.  2.]]
[[ 2.  6.  4.  6.]
 [ 0. -2.  2.  6.]
 [ 0.  0.  2.  2.]]
The relative error ||Ax1-b||/||b|| = 11.707930418441313
The relative distance ||x1-x2||/||x2|| = 24.494895317486026


**1. gauss_elim_v1:** This function performs Gaussian elimination with partial pivoting on a given square matrix A and vector b. It returns the augmented matrix U in row-echelon form. The function also includes an optional print_flag to display intermediate steps of the elimination process.

**2. my_solver_v1:** This function combines gauss_elim_v1 with a back_subs function (assumed to be previously implemented) to solve the linear system Ax = b.